In [1]:
try:
    import ujson as json
    import umap
except ModuleNotFoundError:
    ! pip install ujson -qU
    ! pip install umap-learn -qU
    import ujson as json
    import umap

import requests
import pandas as pd
import random

import xgboost as xgb
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score

if 'google.colab' in str(get_ipython()):
    on_colab = True
else:
    on_colab = False

# xgb_mode = 'gbtree'
xgb_mode = 'dart'


In [ ]:
# Helper function to download files
def download_file(url):
    local_filename = url.split('/')[-1]
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(local_filename, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)
    return local_filename

In [ ]:
# Helper function to pick random parameter from iterable or range
def random_element(parameter_iterable):
    # If it's a tuple treat it as an upper and lower bound
    if isinstance(parameter_iterable, tuple):
        out = random.uniform(parameter_iterable[0], parameter_iterable[1])
        return round(out, 6)

    # If it's a list, return a random element from the list
    elif isinstance(parameter_iterable, list):
        no_choices = len(parameter_iterable)
        return parameter_iterable[random.randrange(0, no_choices)]

    else:
        print('Input not a tuple or list.')
        raise TypeError

In [ ]:
# Dart booster
learning_task_parameters = {'seed': 3}

dart_general_parameters = {'booster': random_element(['gbtree', 'dart']), 'nthread': 2}

gbtree_general_parameters = {'booster': 'gbtree', 'nthread': 2}

def randomize_hparams():
    if xgb_mode == 'dart':
        booster_parameters_ = {'learning_rate': random_element((0.1, 0.8)),
                              'gamma': random_element((0, 5)),
                              'max_depth': random_element(list(range(3,12))),
                              'min_child_weight': random_element((0, 3)),
                              'max_delta_step': 0,
                              'subsample': random_element((0.4, 0.8)),
                              'sampling_method': random_element(['uniform', 'gradient_based']),
                              'lambda': random_element((0.3, 3)),
                              'alpha': random_element((0, 2)),
                              'grow_policy': random_element(['depthwise', 'lossguide']),
                              'max_leaves': random_element([1, 2, 3, 4]),
                              'sample_type': random_element(['uniform', 'weighted']),
                              'normalize_type': random_element(['tree', 'forest']),
                              'rate_drop': random_element((0, 1)),
                              'one_drop': random_element([0, 1]),
                              }

    elif xgb_mode == 'gbtree':
        booster_parameters_ = {'learning_rate': random_element((0.1, 0.8)),
                              'gamma': random_element((0, 5)),
                              'max_depth': random_element(list(range(3,12))),
                              'min_child_weight': random_element((0, 3)),
                              'max_delta_step': 0,
                              'subsample': random_element((0.4, 0.8)),
                              'sampling_method': random_element(['uniform', 'gradient_based']),
                              'lambda': random_element((0.3, 3)),
                              'alpha': random_element((0, 2)),
                              'grow_policy': random_element(['depthwise', 'lossguide']),
                              'max_leaves': random_element([1, 2, 3, 4]),
                              }
    else:
        print('"Global varialbe "xgb_mode" not set!')
        raise TypeError

    if on_colab:
        booster_parameters_.update({'tree_method': 'gpu_hist'})
    else:
        booster_parameters_.update({'tree_method': 'auto'})
    return booster_parameters_

booster_parameters = randomize_hparams()

In [ ]:
# Load x data
if on_colab:
    download_file('https://objectstorage.eu-frankfurt-1.oraclecloud.com/n/frwwzrj6ghal/b/thesis/o/micro_dataset1_resnet18_output_identity.json')
    data_dir = r'micro_dataset1_resnet18_output_identity.json'
    booster_parameters.update({'tree_method': 'gpu_hist'})
else:
    data_dir = r'F:\temp\thesisdata\micro_dataset_1\micro_dataset1_resnet18_output_identity.json'

with open(data_dir, 'r') as f:
    data_dict_list = json.load(f)

data_dict = {}
for element in data_dict_list:
    data_dict.update(element)

# Show first two elements of the dict
# dict(itertools.islice(data_dict.items(), 2))
df_x = pd.DataFrame.from_dict(data_dict, orient='index')
df_x.head()

In [ ]:
# Load y data
if on_colab:
    download_file('https://objectstorage.eu-frankfurt-1.oraclecloud.com/n/frwwzrj6ghal/b/thesis/o/SAATCHI_MICRO_DATASET_PRICE_VIEWSLIKES.tsv')
    data_dir = 'SAATCHI_MICRO_DATASET_PRICE_VIEWSLIKES.tsv'
else:
    data_dir = r'F:\temp\thesisdata\SAATCHI_MICRO_DATASET_PRICE_VIEWSLIKES.tsv'

df_y = pd.read_csv(data_dir, sep='\t')
df_y.set_index('FILENAME', inplace=True)

# Bin the data
# df_y['PRICE_BIN'] = pd.qcut(df_y['PRICE'], q=5)
df_y['PRICE_BIN_IDX'] = pd.qcut(df_y['PRICE'], q=5, labels=[0, 1, 2, 3, 4])
# df_y['LIKES_VIEWS_RATIO_BIN'] = pd.qcut(df_y['LIKES_VIEWS_RATIO'], q=5)
df_y['LIKES_VIEWS_RATIO_BIN_IDX'] = pd.qcut(df_y['LIKES_VIEWS_RATIO'], q=5, labels=[0, 1, 2, 3, 4])
df_y = df_y.astype({'PRICE_BIN_IDX': int, 'LIKES_VIEWS_RATIO_BIN_IDX': int})
df_y.drop(['PRICE', 'LIKES_VIEWS_RATIO'], axis=1, inplace=True)

df_y.head()

In [ ]:
# Join x and y into a single dataframe
df = df_y.join(df_x)
df.head()

In [ ]:
X = df.drop(['PRICE_BIN_IDX', 'LIKES_VIEWS_RATIO_BIN_IDX'], axis=1).values
y = pd.get_dummies(df['LIKES_VIEWS_RATIO_BIN_IDX'].values).to_numpy()
y_idx = df['LIKES_VIEWS_RATIO_BIN_IDX'].values

X_train, X_val, X_test = X[:13000], X[12000:13000], X[13000:]
y_train, y_val, y_test = y[:13000], y[12000:13000], y[13000:]
y_idx_train, y_idx_val, y_idx_test = y_idx[:13000], y_idx[12000:13000], y_idx[13000:]

In [ ]:
# Reduce dimensionality
reducer = umap.UMAP(n_neighbors=30, min_dist=0.0, n_components=50)

mapper = umap.UMAP(n_neighbors=30, min_dist=0.0, n_components=50).fit(X_train, y=y_idx_train)

X_train_embedding = mapper.transform(X_train)
X_test_embedding = mapper.transform(X_test)

In [ ]:
results = []

In [ ]:
# Training
no_iterations = 100
for i in range(no_iterations):
    # create XGBoost instance with default hyper-parameters
    xgb_estimator = xgb.XGBClassifier(objective='binary:logistic',
                                      use_label_encoder=False)

    booster_parameters = randomize_hparams()
    xgb_estimator.set_params(**booster_parameters)

    # create MultiOutputClassifier instance with XGBoost model inside
    multilabel_model = MultiOutputClassifier(xgb_estimator, n_jobs=1)

    multilabel_model.fit(X_train_embedding, y_train)

    accuracy = accuracy_score(y_test, multilabel_model.predict(X_test_embedding)) * 100
    # print(f'Accuracy on test data: {accuracy}')

    r_ = {'run': i,
                    'accuracy': accuracy,
                    'hparams': booster_parameters}

    print(r_)
    results.append(r_)